# Vacuum Agent
### Assignment: 
 ```
We will be using the AIMA Agent code (python) to explore Agents.  The goal is to develop  Vacuum Cleaner Agents living in a world with the following specs.

    All agents can sense dirty and vacuum the area
    The environment is 10 x 10 square
    The environment is static, and is initialized with random dirty locations (prob of dirt specified)
    A Simple Reflex Agent with Random Movement.
    A Simple Reflex Agent with a Bump Sensor and Random Movement
    A Reflect Agent with a bump sensor, and remembers (only) the last two turns.

Testing:

1) For Each agent randomly place them in a room with dirt and count the number of moves required to clean the room (if greater than 1000 moves mark it as a fail)

2) Now, repeat the above trial 1000 times and rank the agents to determine which is best.

Document your project using a Jupyter notebook.
```
### What I know:
 I will be using the energetic dog from AIMA code database located [here](https://github.com/aimacode/aima-python/blob/master/agents.ipynb). This goes over making the 2D enviorment and having dirt spawn randomly in the 2D enviorment aswell as some other things that will be useful for the vacuum agents.

## The Agent
importing the agent class into the code

In [86]:
from agents import *

## The Energetic Dog Code
This is directly taken from the AIMA code database from above this is the basis of our vacuum cleaner agent.

In [87]:
from random import choice, randint

turn = False # global variable to remember to turn if our dog hits the boundary
class EnergeticBlindDog(Agent):
    location = [0,1]
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (ie valid destination location)'''
        global turn
        if not success:
            turn = True # if edge has been reached, remember to turn
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def eat(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Food):
            return True
        return False
    
    def drink(self, thing):
        ''' returns True upon success or False otherwise'''
        if isinstance(thing, Water):
            return True
        return False
        
def program(percepts):
    '''Returns an action based on it's percepts'''
    global turn
    for p in percepts: # first eat or drink - you're a dog!
        if isinstance(p, Food):
            return 'eat'
        elif isinstance(p, Water):
            return 'drink'
    if turn: # then recall if you were at an edge and had to turn
        turn = False
        choice = random.choice((1,2));
    else:
        choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'

This next block of code creates the things to put into the world and the enviorment for the energetic dog

In [88]:
class Food(Thing):
    pass

class Water(Thing):
    pass

class Park(Environment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == "move down":
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.movedown()
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]): #Have the dog eat the first item
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]): #Have the dog drink the first item
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0]) #Delete it from the Park after.

    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

This is a 2D park enviorment which allows all the moves which are legal (It does not allow the dog to move out of bounds) this is the basis for the room that we will use for the vaccum cleaner

In [89]:
class Park2D(XYEnvironment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
            else:
                print('{} decided to move {}wards at location: {}, but couldn\'t'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward(False)
        elif action == "eat":
            items = self.list_things_at(agent.location, tclass=Food)
            if len(items) != 0:
                if agent.eat(items[0]):
                    print('{} ate {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
        elif action == "drink":
            items = self.list_things_at(agent.location, tclass=Water)
            if len(items) != 0:
                if agent.drink(items[0]):
                    print('{} drank {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_edibles = not any(isinstance(thing, Food) or isinstance(thing, Water) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_edibles

This next block of code makes the a 3x3 park enviorment and adds food water and the dog to it then lets the dog make 20 actions

In [90]:
park = Park2D(3,3)
dog = EnergeticBlindDog(program)
dogfood = Food()
water = Water()
park.add_thing(dog, [0,0])
park.add_thing(dogfood, [1,2])
park.add_thing(water, [2,1])
morewater = Water()
park.add_thing(morewater, [0,2])
print("dog started at [0,0], facing down. Let's see if he found any food or water!")
park.run(20)

dog started at [0,0], facing down. Let's see if he found any food or water!
EnergeticBlindDog decided to move downwards at location: [0, 0]
EnergeticBlindDog decided to turnright at location: [0, 1]
EnergeticBlindDog decided to turnleft at location: [0, 1]
EnergeticBlindDog decided to turnleft at location: [0, 1]
EnergeticBlindDog decided to move rightwards at location: [0, 1]
EnergeticBlindDog decided to move rightwards at location: [1, 1]
EnergeticBlindDog drank Water at location: [2, 1]
EnergeticBlindDog decided to move rightwards at location: [2, 1], but couldn't
EnergeticBlindDog decided to turnright at location: [2, 1]
EnergeticBlindDog decided to move downwards at location: [2, 1]
EnergeticBlindDog decided to move downwards at location: [2, 2], but couldn't
EnergeticBlindDog decided to turnright at location: [2, 2]
EnergeticBlindDog decided to turnright at location: [2, 2]
EnergeticBlindDog decided to turnleft at location: [2, 2]
EnergeticBlindDog decided to move leftwards at lo

## Transforming the dog to a vacuum

In [91]:

class Random_Vacuum(Agent):
    """
    This vacuum agent randomly moves around the room looking for dirt if it sense dirt it cleans it up
    """
    location = [0, 1] # randomly places the agent in the room
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (ie valid destination location)'''
        # We do not use the global turn variable here because this agent does not have a bump sensor,
        # the agent can continuely hit walls 
        if not success:
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False

In [92]:
def vacprogram(percepts):
    '''Returns an action based on it's percepts'''
    global turn
    for p in percepts:
        if isinstance(p, Dirt):
            return 'suck'
    choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1:
        return 'turnright'
    elif choice == 2:
        return 'turnleft'
    else:
        return 'moveforward'

In [93]:
class Dirt(Thing):
    pass

In [94]:
class Room2D(XYEnvironment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.R)
        elif action == 'turnleft':
            print('{} decided to {} at location: {}'.format(str(agent)[1:-1], action, agent.location))
            agent.turn(Direction.L)
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            if self.is_inbounds(loc):# move only if the target is a valid location
                print('{} decided to move {}wards at location: {}'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward()
            else:
                print('{} decided to move {}wards at location: {}, but couldn\'t'.format(str(agent)[1:-1], agent.direction.direction, agent.location))
                agent.moveforward(False)
        elif action == "suck":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.suck(items[0]):
                    print('{} cleaned {} at location: {}'
                          .format(str(agent)[1:-1], str(items[0])[1:-1], agent.location))
                    self.delete_thing(items[0])
    
    def count_dirt(self):
        count = 0
        for x in self.things:
            count += 1
        return count-1
            
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_dirt = not any(isinstance(thing, Dirt) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_dirt

In [95]:
room = Room2D(10,10)
vac = Random_Vacuum(vacprogram)
room.add_thing(vac, [randint(0,9),randint(0,9)])
for r in range(0,10):
    for c in range(0,10):
        if random.random() > 0.8:
            print('adding thing')
            room.add_thing(Dirt(),[r,c])
room.run(1000)
if room.is_done():
    print('Success')
else:
    print('Fail')

adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
Random_Vacuum decided to turnleft at location: [7, 9]
Random_Vacuum decided to move rightwards at location: [7, 9]
Random_Vacuum decided to turnleft at location: [8, 9]
Random_Vacuum decided to turnright at location: [8, 9]
Random_Vacuum decided to turnright at location: [8, 9]
Random_Vacuum decided to move downwards at location: [8, 9], but couldn't
Random_Vacuum decided to move downwards at location: [8, 9], but couldn't
Random_Vacuum decided to turnright at location: [8, 9]
Random_Vacuum decided to turnleft at location: [8, 9]
Random_Vacuum decided to turnleft at location: [8, 9]
Random_Vacuum decided to turnright at location: [8, 9]
Random_Vacuum decided to move downwards at location: [8, 9], but couldn't
Random_Vacuum decided to move downwards at location: [8, 9],

# Making the Vaccum better
next we are going to make a vacuum agent which still makes random moves but now has a bump sensor. This should make it so the vacuum does not get stuck on walls.

In [96]:
class Random_Vacuum_B(Agent):
    """
    This vacuum agent randomly moves around the room looking for dirt if it sense dirt it cleans it up also
    includes a  bump sensor.
    """
    location = [0, 1] # randomly places the agent in the room
    direction = Direction("down")
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (ie valid destination location)'''
        global turn
        if not success:
            turn = True # if edge has been reached, remember to turn this is the bump sensor and makes the agent 
            # turn if it hits a wall
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False

In [97]:
room_b = Room2D(10,10)
vac = Random_Vacuum_B(vacprogram)
room_b.add_thing(vac, [randint(0,9),randint(0,9)])
for r in range(0,10):
    for c in range(0,10):
        if random.random() > 0.8:
            print('adding thing')
            room_b.add_thing(Dirt(),[r,c])
room_b.run(1000)
if room_b.is_done():
    print('Success')
else:
    print('Fail')

adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
Random_Vacuum_B decided to move downwards at location: [0, 9], but couldn't
Random_Vacuum_B decided to turnright at location: [0, 9]
Random_Vacuum_B decided to turnright at location: [0, 9]
Random_Vacuum_B decided to move upwards at location: [0, 9]
Random_Vacuum_B decided to turnright at location: [0, 8]
Random_Vacuum_B decided to move rightwards at location: [0, 8]
Random_Vacuum_B decided to move rightwards at location: [1, 8]
Random_Vacuum_B decided to move rightwards at location: [2, 8]
Random_Vacuum_B decided to turnright at location: [3, 8]
Random_Vacuum_B decided to move downwards at location: [3, 8]
Random_Vacuum_B decided to move downwards at location: [3, 9], but couldn't
Random_Vacuum_B decided to turnleft at location: [3, 9]
Random_Vacuum_B decided to move 

Random_Vacuum_B decided to move leftwards at location: [3, 4]
Random_Vacuum_B decided to move leftwards at location: [2, 4]
Random_Vacuum_B decided to turnright at location: [1, 4]
Random_Vacuum_B decided to turnright at location: [1, 4]
Random_Vacuum_B decided to move rightwards at location: [1, 4]
Random_Vacuum_B decided to move rightwards at location: [2, 4]
Random_Vacuum_B decided to move rightwards at location: [3, 4]
Random_Vacuum_B decided to move rightwards at location: [4, 4]
Random_Vacuum_B decided to move rightwards at location: [5, 4]
Random_Vacuum_B decided to turnleft at location: [6, 4]
Random_Vacuum_B decided to move upwards at location: [6, 4]
Random_Vacuum_B decided to turnright at location: [6, 3]
Random_Vacuum_B decided to move rightwards at location: [6, 3]
Random_Vacuum_B decided to move rightwards at location: [7, 3]
Random_Vacuum_B decided to move rightwards at location: [8, 3]
Random_Vacuum_B decided to turnright at location: [9, 3]
Random_Vacuum_B decided to t

Now we are going to allow the vaccum to remember 2 moves and seeing if this will help the vacuum clean better. I have the vacuum program remeber the last 2 moves picked and and make sure it does not do moves which just undo the moves it just did. this should really improve the efficiency of the vacuum as it ownt get stuck rotating in one spot.

In [98]:
class Random_Vacuum_B_2(Agent):
    """
    This vacuum agent randomly moves around the room looking for dirt if it sense dirt it cleans it up also
    includes a  bump sensor.
    """
    location = [0, 1] # randomly places the agent in the room
    direction = Direction("down")
    
    
    def moveforward(self, success=True):
        '''moveforward possible only if success (ie valid destination location)'''
        global turn
        if not success:
            turn = True # if edge has been reached, remember to turn this is the bump sensor and makes the agent 
            # turn if it hits a wall
            return
        if self.direction.direction == Direction.R:
            self.location[0] += 1
        elif self.direction.direction == Direction.L:
            self.location[0] -= 1
        elif self.direction.direction == Direction.D:
            self.location[1] += 1
        elif self.direction.direction == Direction.U:
            self.location[1] -= 1
    
    def turn(self, d):
        self.direction = self.direction + d
        
    def suck(self, thing):
        '''returns True upon success or False otherwise'''
        if isinstance(thing, Dirt):
            return True
        return False

Below is the new vacuum program which has a new list which we append remove the first move then append the new move on this allows us to keep track of the last 2 move which can help make better decisions.

In [99]:
moves = [1,2]
def vacprogram_2(percepts):
    '''Returns an action based on it's percepts'''
    global turn
    global moves 
    for p in percepts:
        if isinstance(p, Dirt):
            moves.pop(0)
            moves.append('suck')
            return 'suck'
    choice = random.choice((1,2,3,4)) # 1-right, 2-left, others-forward
    if choice == 1 and 'turnleft' not in moves:
        moves.pop(0)
        moves.append('turnright')
        return 'turnright'
    elif choice == 2 and 'turnright' not in moves:
        moves.pop(0)
        moves.append('turnleft')
        return 'turnleft'
    else:
        moves.pop(0)
        moves.append('moveforward')
        return 'moveforward'

In [100]:
room_2 = Room2D(10,10)
vac = Random_Vacuum_B_2(vacprogram_2)
room_2.add_thing(vac, [randint(0,9),randint(0,9)])
for r in range(0,10):
    for c in range(0,10):
        if random.random() > 0.8:
            print('adding thing')
            room_2.add_thing(Dirt(),[r,c])
room_2.run(1000)
if room_2.is_done():
    print('Success')
else:
    print('Fail')
    
print(room_2.count_dirt())

adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
adding thing
Random_Vacuum_B_2 decided to turnleft at location: [5, 9]
Random_Vacuum_B_2 decided to move rightwards at location: [5, 9]
Random_Vacuum_B_2 decided to move rightwards at location: [6, 9]
Random_Vacuum_B_2 decided to turnright at location: [7, 9]
Random_Vacuum_B_2 decided to move downwards at location: [7, 9], but couldn't
Random_Vacuum_B_2 decided to move downwards at location: [7, 9], but couldn't
Random_Vacuum_B_2 decided to turnright at location: [7, 9]
Random_Vacuum_B_2 decided to turnright at location: [7, 9]
Random_Vacuum_B_2 decided to move upwards at location: [7, 9]
Random_Vacuum_B_2 decided to turnright at location: [7, 8]
Random_Vacuum_B_2 decided to move rightwards at location: [

# Testing
The following is to ttes the agents against each other and to see how they preform. As above each agent will get 1000 moves in the test enviorment to clean the room up after that the test wil be a failure each failure will record how many pieces of dirt are left and at the end the average will be printed. the test will occur 1000 times for each agent.

First I made a new XYEnviornmet to remove the print lines as running the test 1000 times will clutter instead a final print of the average dirt left the number of fails and successes that were recorded will appear.

In [101]:
class Room2DTest(XYEnvironment):
    def percept(self, agent):
        '''prints & return a list of things that are in our agent's location'''
        things = self.list_things_at(agent.location)
        return things
    
    def execute_action(self, agent, action):
        '''changes the state of the environment based on what the agent does.'''
        if action == 'turnright':
            agent.turn(Direction.R)
        elif action == 'turnleft':
            agent.turn(Direction.L)
        elif action == 'moveforward':
            loc = copy.deepcopy(agent.location) # find out the target location
            if agent.direction.direction == Direction.R:
                loc[0] += 1
            elif agent.direction.direction == Direction.L:
                loc[0] -= 1
            elif agent.direction.direction == Direction.D:
                loc[1] += 1
            elif agent.direction.direction == Direction.U:
                loc[1] -= 1
            if self.is_inbounds(loc):# move only if the target is a valid location
                agent.moveforward()
            else:
                agent.moveforward(False)
        elif action == "suck":
            items = self.list_things_at(agent.location, tclass=Dirt)
            if len(items) != 0:
                if agent.suck(items[0]):
                    self.delete_thing(items[0])
    
    def count_dirt(self):
        count = 0
        for x in self.things:
            count += 1
        return count-1
            
                    
    def is_done(self):
        '''By default, we're done when we can't find a live agent, 
        but to prevent killing our cute dog, we will stop before itself - when there is no more food or water'''
        no_dirt = not any(isinstance(thing, Dirt) for thing in self.things)
        dead_agents = not any(agent.is_alive() for agent in self.agents)
        return dead_agents or no_dirt

Next we will run the test for each in a seperate code block

In [102]:
total_dirt = 0
success = 0 
for i in range(1000): #runs the program 1000 times 
    room_test = Room2DTest(10,10)
    vac_1 = Random_Vacuum(vacprogram)
    room_test.add_thing(vac_1, [randint(0,9),randint(0,9)])
    for r in range(0,10):
        for c in range(0,10):
            if random.random() > 0.8:
                room_test.add_thing(Dirt(),[r,c])
    room_test.run(1000)
    if room_test.is_done():
        success += 1
    total_dirt += room_test.count_dirt()
    
print("""The random vacuum ran 1000 times it sucessfully cleaned the room {} times the total dirt it did not clean up was {} thats an average of {} pieces of dirt""".format(success, total_dirt, total_dirt/1000))

The random vacuum ran 1000 times it sucessfully cleaned the room 328 times the total dirt it did not clean up was 1358 thats an average of 1.358 pieces of dirt


In [103]:
total_dirt_b = 0
success_b = 0 
for i in range(1000): #runs the program 1000 times 
    room_test = Room2DTest(10,10)
    vac_2 = Random_Vacuum_B(vacprogram)
    room_test.add_thing(vac_2, [randint(0,9),randint(0,9)])
    for r in range(0,10):
        for c in range(0,10):
            if random.random() > 0.8:
                room_test.add_thing(Dirt(),[r,c])
    room_test.run(1000)
    if room_test.is_done():
        success_b += 1
    total_dirt_b += room_test.count_dirt()
    
print("""The random vacuum with bump sensor ran 1000 times it sucessfully cleaned the room {} times the total dirt it did not clean up was {} thats an average of {} pieces of dirt""".format(success_b, total_dirt_b, total_dirt_b/1000))

The random vacuum with bump sensor ran 1000 times it sucessfully cleaned the room 349 times the total dirt it did not clean up was 1255 thats an average of 1.255 pieces of dirt


In [104]:
total_dirt_b_2 = 0
success_b_2 = 0 
for i in range(1000): #runs the program 1000 times 
    room_test = Room2DTest(10,10)
    vac_3 = Random_Vacuum_B_2(vacprogram_2)
    room_test.add_thing(vac_3, [randint(0,9),randint(0,9)])
    for r in range(0,10):
        for c in range(0,10):
            if random.random() > 0.8:
                room_test.add_thing(Dirt(),[r,c])
    room_test.run(1000)
    if room_test.is_done():
        success_b_2 += 1
    total_dirt_b_2 += room_test.count_dirt()
    
print("""The random vacuum with bump sensor and 2 move memory ran 1000 times it sucessfully cleaned the room {} times the total dirt it did not clean up was {} thats an average of {} pieces of dirt""".format(success_b_2, total_dirt_b_2, total_dirt_b_2/1000))

The random vacuum with bump sensor and 2 move memory ran 1000 times it sucessfully cleaned the room 542 times the total dirt it did not clean up was 716 thats an average of 0.716 pieces of dirt


In [105]:
print(
"""
Vacuum Type : Success : Failure : Total Dirt : Average Dirt
Random Vac  : {}     : {}     : {}       : {}
Bump Vacuum : {}     : {}     : {}       : {}
2 Move Mem  : {}     : {}     : {}        : {}
""".format(success, 1000- success, total_dirt, total_dirt/1000,
           success_b, 1000- success_b, total_dirt_b, total_dirt_b/1000,
           success_b_2, 1000- success_b_2, total_dirt_b_2, total_dirt_b_2/1000
)
)


Vacuum Type : Success : Failure : Total Dirt : Average Dirt
Random Vac  : 328     : 672     : 1358       : 1.358
Bump Vacuum : 349     : 651     : 1255       : 1.255
2 Move Mem  : 542     : 458     : 716        : 0.716



#### Conclusion
To conclude I think that it was really intresting to se that there was not much of a change from the bump sensor and non bump sensor vacuums. I was also suprised by the jump that remembering 2 moves gave the agent. This is really made me think about how much precepts an agent needs to be successful it also made me think about hte way we categorize sucesss since besides the 2 move memory agent failed a majority of the time.